In [1]:
import pandas as pd
import numpy as np
from numpy import dstack
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM, GRU, Bidirectional
from keras.models import load_model
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
import tensorflow_probability as tfp
import requests
from sklearn.preprocessing import MinMaxScaler
pd.options.mode.chained_assignment = None
tfd = tfp.distributions
tfk = tf.keras


In [62]:
train = pd.read_csv('/content/강릉_knn_train.csv', index_col = 'yyyymmddhhnn')

In [60]:
len(train['2020-01-01 00:00:00':'2020-05-31 23:50:00'])

21888

In [63]:
scaler = MinMaxScaler()
scale_cols = ['solarza','esr','uv']
df_scaled = scaler.fit_transform(train[scale_cols])

df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

In [64]:
TEST_SIZE = 25345

dataset = df_scaled
train = df_scaled[:-TEST_SIZE]
test = df_scaled[-TEST_SIZE:]

In [65]:
def make_dataset(data, label, window_size):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [68]:
def gru(train, test, feature_cols, label_cols, window_size, units):

    train_feature = train[feature_cols]
    train_label = train[label_cols]

    # train dataset
    train_feature, train_label = make_dataset(train_feature, train_label, window_size)

    # train, validation set 생성
    x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

    x_train.shape, x_valid.shape
    # ((6086, 20, 4), (1522, 20, 4))

    test_feature = test[feature_cols]
    test_label = test[label_cols]

    # test dataset (실제 예측 해볼 데이터)
    test_feature, test_label = make_dataset(test_feature, test_label, window_size)
    test_feature.shape, test_label.shape
    # ((180, 20, 4), (180, 1))

    model = Sequential()
    model.add(GRU(units,input_shape=(train_feature.shape[1], train_feature.shape[2]), return_sequences=False ) )
    model.add(Dropout(0.2))
    # model.add(GRU(units = 64,return_sequences=True))
    # model.add(Dropout(0.2))
    # model.add(GRU(units = 32))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    filename = os.path.join('./', 'gru.h5')
    checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    
    GRU_history = model.fit(x_train, y_train, 
                        epochs=100, 
                        batch_size=16,
                        validation_data=(x_valid, y_valid), 
                        callbacks=[early_stop, checkpoint])

    # weight 로딩
    model.load_weights(filename)

    # prediction
    train_pred = model.predict(train_feature)
    test_pred = model.predict(test_feature)

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(train_pred, train_label))
    print('Train Score: %.3f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(test_pred, test_label))
    print('Test Score: %.3f RMSE' % (testScore))
    
    visualizer(test_label, test_pred, "GRU Prediction")

def bilstm(train, test, feature_cols, label_cols, window_size, units):


    train_feature = train[feature_cols]
    train_label = train[label_cols]

    # train dataset
    train_feature, train_label = make_dataset(train_feature, train_label, window_size)

    # train, validation set 생성
    x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

    x_train.shape, x_valid.shape
    # ((6086, 20, 4), (1522, 20, 4))

    test_feature = test[feature_cols]
    test_label = test[label_cols]

    # test dataset (실제 예측 해볼 데이터)
    test_feature, test_label = make_dataset(test_feature, test_label, window_size)
    test_feature.shape, test_label.shape
    # ((180, 20, 4), (180, 1))


    model = Sequential()
    model.add(Bidirectional(LSTM(units = units,                             
              return_sequences=False),
              input_shape=(x_train.shape[1], x_train.shape[2])))
    # model.add(Dropout(0.2))
    # model.add(Bidirectional(LSTM(units = 24,return_sequences=True)))
    # model.add(Dropout(0.2))
    # model.add(Bidirectional(LSTM(units = 16)))

    model.add(Dense(1))

    #Compile model
    model.compile(loss='mse', optimizer='adam')
  
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    filename = os.path.join('./', 'bilstm.h5')
    checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    lstm_history = model.fit(x_train, y_train, 
                        epochs=100, 
                        batch_size=32,
                        validation_data=(x_valid, y_valid), 
                        callbacks=[early_stop, checkpoint])

    # weight 로딩
    model.load_weights(filename)

    # 예측

    train_pred = model.predict(train_feature)
    test_pred = model.predict(test_feature)

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(train_pred, train_label))
    print('Train Score: %.3f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(test_pred, test_label))
    print('Test Score: %.3f RMSE' % (testScore))

    visualizer(test_label, test_pred, "B-LSTM Prediction")
    
def lstm(train, test, feature_cols, label_cols, window_size, units):

    train_feature = train[feature_cols]
    train_label = train[label_cols]

    # train dataset
    train_feature, train_label = make_dataset(train_feature, train_label, window_size)

    # train, validation set 생성
    x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)

    x_train.shape, x_valid.shape
    # ((6086, 20, 4), (1522, 20, 4))

    test_feature = test[feature_cols]
    test_label = test[label_cols]

    # test dataset (실제 예측 해볼 데이터)
    test_feature, test_label = make_dataset(test_feature, test_label, window_size)
    test_feature.shape, test_label.shape
    # ((180, 20, 4), (180, 1))

    model = Sequential()
    model.add(LSTM(units, 
                  input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                  return_sequences=False)
              )
    # model.add(Dropout(0.2))
    # model.add(LSTM(64, return_sequences=True))
    # model.add(Dropout(0.2))
    # model.add(LSTM(32, return_sequences=False))


    model.add(Dense(1))
    
    model.compile(loss='mse', optimizer='adam')
    early_stop = EarlyStopping(monitor='val_loss', patience=5)
    filename = os.path.join('./', 'lstm.h5')
    checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

    lstm_history = model.fit(x_train, y_train, 
                        epochs=100, 
                        batch_size=16,
                        validation_data=(x_valid, y_valid), 
                        callbacks=[early_stop, checkpoint])

    # weight 로딩
    model.load_weights(filename)

    # 예측
    train_pred = model.predict(train_feature)
    test_pred = model.predict(test_feature)

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(train_pred, train_label))
    print('Train Score: %.3f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(test_pred, test_label))
    print('Test Score: %.3f RMSE' % (testScore))

    visualizer(test_label, test_pred, "LSTM Prediction")

def load_all_models():
    models = ['lstm', 'gru', 'bilstm']
    #models = ['bilstm', 'gru']


    all_models = []
    for model in models:
        filename = model + '.h5'
        model = load_model(filename)
        all_models.append(model)
    return all_models

# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(inputX):
    stackX = None
    models = load_all_models()
    for model in models:
      # make prediction
      yhat = model.predict(inputX, verbose=0)
      # stack predictions into [rows, members, probabilities]
      if stackX is None:
        stackX = yhat
      else:
        stackX = dstack((stackX, yhat))


    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(inputX, inputy, window_size):


    # create dataset using ensemble
    stackedX = stacked_dataset(inputX)
    # fit standalone model
    model = KNeighborsRegressor(n_neighbors=3)
    model.fit(stackedX, inputy[window_size:])
    return model

# make a prediction with the stacked model
def stacked_prediction(inputX, inputy, window_size):

    test_feature, test_label = make_dataset(inputX, inputy, window_size)
    model = fit_stacked_model(test_feature, inputy, window_size)
    # create dataset using ensemble
    stackedX = stacked_dataset(test_feature)
    # make a prediction
    ensemble_pred = model.predict(stackedX)

    # calculate root mean squared error
    testScore = math.sqrt(mean_squared_error(ensemble_pred, test_label))
    print('Test Score: %.3f RMSE' % (testScore))

    visualizer(test_label, ensemble_pred, "Ensemble Prediction")



def visualizer(test_label, pred, title):
    fig, ax = plt.subplots()
    plt.title('Comparison Between Actual Price and ' + title +' Price')
    plt.ylabel('Price')
    plt.xlabel('Date')
    
    plt.plot(test_label, color='r', label='Actual Price')
    plt.plot(pred, color='b', label='Prediction Price')
    
    ax.grid(True)
    fig.tight_layout()
    plt.legend()
    plt.savefig(title + '.svg', format='svg')
    plt.show()

def run_all(train, test, feature_cols, label_cols, window_size, units):
    lstm(train, test, feature_cols, label_cols, window_size, units)
    gru(train, test, feature_cols, label_cols, window_size, units)
    bilstm(train, test, feature_cols, label_cols, window_size, units)
    stacked_prediction(test[feature_cols], test[label_cols], window_size)

In [ ]:
x_only = ['solarza','esr']
feature_cols = x_only
label_cols = ['uv']

run_all(train, test, feature_cols, label_cols, 10, 256)

Epoch 1/100
3995/3996 [============================>.] - ETA: 0s - loss: 0.0062
Epoch 1: val_loss improved from inf to 0.00537, saving model to ./lstm.h5
3996/3996 [==============================] - 121s 30ms/step - loss: 0.0062 - val_loss: 0.0054
Epoch 2/100
3994/3996 [============================>.] - ETA: 0s - loss: 0.0056
Epoch 2: val_loss improved from 0.00537 to 0.00523, saving model to ./lstm.h5
3996/3996 [==============================] - 120s 30ms/step - loss: 0.0056 - val_loss: 0.0052
Epoch 3/100
3996/3996 [==============================] - ETA: 0s - loss: 0.0056
Epoch 3: val_loss did not improve from 0.00523
3996/3996 [==============================] - 119s 30ms/step - loss: 0.0056 - val_loss: 0.0054
Epoch 4/100
3996/3996 [==============================] - ETA: 0s - loss: 0.0055
Epoch 4: val_loss improved from 0.00523 to 0.00517, saving model to ./lstm.h5
3996/3996 [==============================] - 121s 30ms/step - loss: 0.0055 - val_loss: 0.0052
Epoch 5/100
3995/3996 [====